<a href="https://colab.research.google.com/github/yuukun123/Python/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [4]:
# Tùy chỉnh để hiển thị biểu đồ đẹp hơn
sns.set(style="whitegrid")

In [10]:
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
# khai báo biến với file dữ liệu
data1 = "/content/drive/MyDrive/febrl3_UNSW_rep.csv"
df = pd.read_csv(data1)

In [12]:
# xem thông tin dữ liệu
print(df.info())
# xem những hàng đầu tiên
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     5000 non-null   int64  
 1   rec_id         5000 non-null   object 
 2   given_name     4844 non-null   object 
 3   surname        4921 non-null   object 
 4   street_number  5000 non-null   int64  
 5   address_1      4846 non-null   object 
 6   address_2      4307 non-null   object 
 7   suburb         4915 non-null   object 
 8   postcode       5000 non-null   int64  
 9   state          4915 non-null   object 
 10  day            4845 non-null   float64
 11  month          4845 non-null   float64
 12  year           4845 non-null   float64
 13  match_id       5000 non-null   int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 547.0+ KB
None
   Unnamed: 0      rec_id given_name   surname  street_number  \
0           0        1496   mitchell     green         

In [16]:
new_df = df.dropna()

new_df.head(30)

,Unnamed: 0,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,day,month,year,match_id
0,0,1496,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,1.0,1.0,1970.0,1496
1,1,552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,1.0,1.0,1970.0,552
2,2,988-dup-1,madeline,mason,54,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,1.0,1.0,1970.0,988
5,5,193,toby,webb,6,kent street,kersey' south,towradgi,3107,nsw,1.0,1.0,1970.0,193
7,7,1695-dup-3,lydia,mccattmhy,32,mckillop ecircuit,telemon orchards,corio,4670,nsw,1.0,1.0,1970.0,1695
9,9,1778-dup-1,dylan,zilm,17,blameycrescent,condinup,wattle glen,4211,vic,1.0,1.0,1970.0,1778
11,11,382-dup-0,isabella,vincent,16,wilshiredstreet,erinsavle,rose ay,6207,vic,1.0,1.0,1970.0,382
13,13,1671-dup-0,connor,shepherd,39,burnage place,rowethorpe,winstonhills,3183,nsw,1.0,1.0,1970.0,1671
14,14,1816-dup-1,natalia,gredn,358,macrossan crescent,newman medicajl centre,yagoona,2535,qld,1.0,1.0,1970.0,1816
15,15,459-dup-4,georgia,pettigrove,5,sinclair street,pinds,prairei,2478,nsw,1.0,1.0,1970.0,459


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df.to_csv('/content/drive/MyDrive/titanic.csv', index=False)

Mounted at /content/drive
